# Primeiro projeto de Automação Web e webscraping com Python

Coletar na internet, de forma automática, a cotação e saber quanto deverá cobrar pelos produtos, considerando uma margem de contribuição na base de dados.

Para isso, vou criar uma automação web:

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# abrir um navegador
navegador = webdriver.Chrome()

navegador.get("https://www.google.com/")

#Pegando a cotação do Dólar
navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")

navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_dolar = navegador.find_element(By.XPATH,
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value") 
print(cotacao_dolar)

#Pegando a cotação do Euro
navegador.get("https://www.google.com/")
navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element(By.XPATH,
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(cotacao_euro)

#Pegando a cotação do Ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")

cotacao_ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute("value")
cotacao_ouro = cotacao_ouro.replace(",", ".")
print(cotacao_ouro)

navegador.quit()

5.2585999999999995
5.548007051
309.41



- Importando a base de dados

In [4]:
#Importando a lista de produtos
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Macbook,1199.99,Dólar,5,5999.95,1.70,10199.915
1,Apple earbuds,112.99,Euro,6,677.94,1.70,1152.498
2,Samsung S22,699.99,Dólar,5,3499.95,1.70,5949.915
3,IPhone 13,799.00,Dólar,5,3995.00,1.70,6791.500
4,Fone Razer,69.99,Euro,6,419.94,1.90,797.886
5,Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Colar 20g,20.00,Ouro,350,7000.00,1.15,8050.000
7,Bicicleta,150.00,Dólar,5,750.00,1.15,862.500
8,Mac,900.00,Dólar,5,4500.00,1.15,5175.000
9,Câmera Sony,999.99,Dólar,5,4999.95,1.70,8499.915


- Atualizando os preços e o cálculo do Preço Final

In [6]:
# Recalcular o preço de cada produto
# atualizar a cotação

tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

# atualizar o preço base reais (preço base original * cotação)
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

# atualizar o preço final (preço base reais * Margem)
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

tabela["Preço de Venda"] = tabela["Preço de Venda"].map("R${:.2f}".format)

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Macbook,1199.99,Dólar,5.258600,6310.267414,1.70,R$10727.45
1,Apple earbuds,112.99,Euro,5.548007,626.869317,1.70,R$1065.68
2,Samsung S22,699.99,Dólar,5.258600,3680.967414,1.70,R$6257.64
3,IPhone 13,799.00,Dólar,5.258600,4201.621400,1.70,R$7142.76
4,Fone Razer,69.99,Euro,5.548007,388.305013,1.90,R$737.78
5,Xiaomi,480.48,Dólar,5.258600,2526.652128,2.00,R$5053.30
6,Colar 20g,20.00,Ouro,309.410000,6188.200000,1.15,R$7116.43
7,Bicicleta,150.00,Dólar,5.258600,788.790000,1.15,R$907.11
8,Mac,900.00,Dólar,5.258600,4732.740000,1.15,R$5442.65
9,Câmera Sony,999.99,Dólar,5.258600,5258.547414,1.70,R$8939.53


### Exportar a nova base de preços atualizada

In [7]:
#Salvar os novos preços dos produtos
tabela.to_excel("Produtos Atualizado.xlsx", index=False)